# Funciones Basicas de TensorFlow

## Introduccion

TensorFlow propone un modelo computacional distinto al normalmente usado: la computacion simbolica. Mas adelante veremos que significa esto, pero por ahora solo veamos un ejemplo de una computacion en python estadar

In [68]:
w = 1.0  #parametro
b = 2.0  #parametro

def f(x):  # x es un input
    return w * x + b

f(1) # w * x + b == 1.0 * 1.0 + 2.0 == 3.0

3.0

Aqui diremos que `f` es una funcion de `x` que esta parametrizada por las variables `w` y `b`. Esto lo podriamos escribir matematicamente de la siguiente forma:

# $f(x; w,b) = w x + b$

Esto quiere decir que si bien `w` y `b` no son parte de los argumentos de `f`, si determinan su comportamiento. Si cambiamos `w` o `b` tambien cambiara el valor de `f` dado cierto `x`. Modifiquemos `b` a `10.0` a ver que sucede:

In [76]:
b = 10.0

f(1) # w * x + b == 1.0 * 1.0 + 10.0 == 11.0

11.0

Como es de esperarse, el valor de `f` cambio. Durante el resto del notebook veremos lo necesario para recrear esto en TensorFlow

## Import

Por convension se importa `tensorflow` como `tf`

In [18]:
import tensorflow as tf

## Constants
El tensor mas simple de TensorFlow es un tensor constante. Para crear uno utilizamos `tf.constant`

In [78]:
c = tf.constant(5.0)

print c

Tensor("Const_4:0", shape=(), dtype=float32)


Sin embargo, como vemos en la evaluacion de la celda anterior, este tensor constante `c` no parece indicarnos el valor que contiene, para extraer el valor primero tenemos crear crear una session y correr el tensor en ella:

## Session

In [38]:
sess = tf.Session()

sess.run(c)

5.0

Utilizando `run` pudimos recuperar el valor de `c`! Sin embargo es no parece muy utilil pare ahora. Sin embargo, veamos como crear operaciones un poco mas complejas. Al igual que en numpy, los tensores sobreescriben la mayoria de los operadores de python y nos permiten crear nuevos tensores de esta manera:

In [42]:
k = c * 12.0

print k

Tensor("mul_2:0", shape=(), dtype=float32)


Aqui creamos `k` a partir de `c` y el numero `12.0`. Lo interesante es que el `print` nos da una pista de que `k` es un tensor de tipo "mul", esto se debe a que representa la operacion de multiplicacion. Para recuperar el valor de `k` debemos ejecutarlo en la session denuevo.

In [43]:
sess.run(k)

60.0

## Placeholder
Hasta ahora solo hemos creado expresiones constantes y esto no es tan util. Para introducir entradas del exterior a las expresiones debemos crear un tensor tipo `placeholder`

In [71]:
x = tf.placeholder(tf.float32, ())
y = x * 2.0

print y

Tensor("mul_3:0", shape=(), dtype=float32)


Ok, definimos un tensor sin ningun valor y creamos un operacion a partir de el, ¿pero que pasara si lo evaluamos en la session?

In [79]:
try:
    sess.run(y)
except:
    print "Error!"

Error!


Como vemos obtenemos un error, esto es porque tensorflow necesita que definamos un valor para `x` dentro del grafo para poder calcular el valor de `y`. Esto lo hacemos pasando un diccionario de tensores a valores al parametro `feed_dict` de `run`

In [75]:
valores = {x:  9.0}

print float(sess.run(y, feed_dict=valores))

18.0


## Variable
Hemos creados expresiones "puras" en el sentido que dados los mismo valores para los `placeholders` siempre data los mismo valores para el resto de los tensores, en otras palabras, no hay estado. Para tener sistemas que guarden estado utilizamos `Variable`. Creemos una variable sensilla

In [83]:
a = tf.Variable(1.0)
print a

Tensor("Variable_3/read:0", shape=(), dtype=float32)


Aqui creamos una variable con un valor inicial de `1.0`, sin embargo las variables son un poco diferentes al resto de los tensores en el sentido que deben ser inicializadas, esto posiblemente es para reservar memoria en el dispositivo que vaya a almacenar esta variable, si no lo hacemos e intentamos evaluar una expresion tendremos errores

In [84]:
try:
    print sess.run(a)
except:
    print "Error!"

Error!


Para iniciar las variables corremos `tf.global_variables_initializer()`

In [85]:
init = tf.global_variables_initializer()
sess.run(init)

Esta operacion no retorna nada pero como efecto secundario inicia las variables. Ahora si corremos `a` podemos obtener su valor

In [88]:
sess.run(a)

1.0

Como dijimos el proposito de tener variables es poder almacener y lo mas importante actualizar cierto estado, de otra manera parecen constantes. Por lo tanto vamos a definir la operacion

$a := a * 2.0$

en TensorFlow

In [89]:
update = tf.assign(a, a * 2.0)

Ahora si corremos `a` veremos que

In [91]:
sess.run(a)

1.0

...sigue igual. Esto es porque debemos correr el tensor `update` para que `a` se actualice

In [93]:
sess.run(update)

print sess.run(a)

4.0


Perfecto! Si volvemos a correr la celda anterior varias veces veremos que `a` se va duplicando cada vez. 

## Uniendo Todo
Ahora estamos preparados para recrear nuestra funcion `f` original en tensorflow

In [94]:
w = tf.Variable(1.0) #parametro
b = tf.Variable(2.0) #parametro

x = tf.placeholder(tf.float32, []) #input

f = w * x + b  # f(x; w, b)

#iniciar variables
sess.run(tf.global_variables_initializer())

# f(1.0)
sess.run(f, feed_dict={x: 1.0})

3.0

Genial! Reproducimos el resultado original de python. Ahora nos falta modificar `b` a `10.0` como lo hicimos en anteriormente y ver como cambia `f`

In [98]:
# b = 10.0
sess.run(tf.assign(b, 10.0))

# f(1.0)
sess.run(f, feed_dict={x: 1.0})

11.0

## ¿Que sigue?

Todo esto realmente parece una forma muy complicada de realizar oparaciones que serian mas facil con python o numpy. Las ventajas de TensorFlow para el Deep Learning y la computacion cientifica en general son las siguientes:

* Cada tensor puede estar ubicado en un dispositivo diferente como una gpu, la cpu normal, o una maquina remota. Esto es importante para correr grandes modelos utilizando paralelismo y computacion distribuida.
* La computacion simbolica le permite a tensorflow calcular derivadas de una expresion con respecto a una subexpresion de la misma, esto es importante para calcular el gradiente de las redes neuronales.

En el proximo notebook veremos como calcular derivadas con tensorflow!